<a href="https://colab.research.google.com/github/GabrielKP/continual-learning/blob/main/comp_mean_std_cnn_cifar100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

import random
import math
import time
import copy
import itertools

### Load pretrained convLayers

For details look at pre_train_on_cifar10.ipynb

#### convLayers

In [ ]:
class ConvLayers(nn.Module):
    """
    5 torch convolutional layers with batchnormalization
    and ReLU nonlinearity. No pooling is used. Inherits from nn.Module.
    Requires images with the same height and width.

    Methods
    -------
    forward(X) -> None
        Applies convolutions on input X

    Attributes
    ----------
    img_size : int
        number indicating the height of input images
    in_channels : int
        channels of input data
    out_channels : int
        amount of channels for last layer
    out_size : int
        height of the output data
    out_units : int
        total amount of units outputting
    """

    def __init__(self, in_channels, img_size):
        """
        Creates 5 conv layers with batchnorm and ReLU activation

        Parameters
        ----------
        in_channels : int
            Amount of channels the input data has
        img_size : int
            Amount of pixels in one axis
        """
        super(ConvLayers, self).__init__()

        list_out_channels = [16,32,64,128,256]

        self.img_size = img_size
        self.out_size = int(math.ceil(img_size / 2**4))
        self.in_channels = in_channels
        self.out_channels = list_out_channels[-1]
        self.out_units = self.out_channels * self.out_size**2
        
        self.conv1 = nn.Conv2d(in_channels, list_out_channels[0],
                               kernel_size=3, stride=1, bias=True, padding=1)
        self.bn1 = nn.BatchNorm2d(list_out_channels[0])
        self.nl1 = nn.ReLU()

        self.conv2 = nn.Conv2d(list_out_channels[0], list_out_channels[1],
                               kernel_size=3, stride=2, bias=True, padding=1)
        self.bn2 = nn.BatchNorm2d(list_out_channels[1])
        self.nl2 = nn.ReLU()
        
        self.conv3 = nn.Conv2d(list_out_channels[1], list_out_channels[2],
                               kernel_size=3, stride=2, bias=True, padding=1)
        self.bn3 = nn.BatchNorm2d(list_out_channels[2])
        self.nl3 = nn.ReLU()

        self.conv4 = nn.Conv2d(list_out_channels[2], list_out_channels[3],
                               kernel_size=3, stride=2, bias=True, padding=1)
        self.bn4 = nn.BatchNorm2d(list_out_channels[3])
        self.nl4 = nn.ReLU()

        self.conv5 = nn.Conv2d(list_out_channels[3], list_out_channels[4],
                               kernel_size=3, stride=2, bias=True, padding=1)
        self.bn5 = nn.BatchNorm2d(list_out_channels[4])
        self.nl5 = nn.ReLU()

    def forward(self, X):
        """
        Applies convolutions on input X
        """

        X1 = self.nl1(self.bn1(self.conv1(X)))
        X2 = self.nl2(self.bn2(self.conv2(X1)))
        X3 = self.nl3(self.bn3(self.conv3(X2)))
        X4 = self.nl4(self.bn4(self.conv4(X3)))
        X5 = self.nl5(self.bn5(self.conv5(X4)))

        return X5

#### Get pre-trained model from github and load it in

In [ ]:
!wget -O convLayers_trained_cifar10.pt https://github.com/GabrielKP/continual-learning/blob/main/convLayers_trained_cifar10.pt?raw=true

--2021-10-10 11:02:27--  https://github.com/GabrielKP/continual-learning/blob/main/convLayers_trained_cifar10.pt?raw=true
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/GabrielKP/continual-learning/raw/main/convLayers_trained_cifar10.pt [following]
--2021-10-10 11:02:28--  https://github.com/GabrielKP/continual-learning/raw/main/convLayers_trained_cifar10.pt
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/GabrielKP/continual-learning/main/convLayers_trained_cifar10.pt [following]
--2021-10-10 11:02:28--  https://raw.githubusercontent.com/GabrielKP/continual-learning/main/convLayers_trained_cifar10.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubu

In [ ]:
convLayers = torch.load("convLayers_trained_cifar10.pt")

## Parameters

In [ ]:
BATCH_SIZE = 64

## Data

norms and means computed in `pre_train_on_cifar10.ipnyb`

In [ ]:
means = [0.491400808095932, 0.48215898871421814, 0.44653093814849854]
stds = [0.24703224003314972, 0.24348513782024384, 0.26158785820007324]

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(means, stds)])

ds_train_cifar100 = datasets.CIFAR100("data", train=True,
                                      transform=transform,
                                      download=True)

dl_train_cifar100 = DataLoader(ds_train_cifar100, batch_size=BATCH_SIZE,
                              shuffle=True)


  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data


# Compute mean and std

In [ ]:
outputs = torch.zeros((len(dl_train_cifar100) * BATCH_SIZE,
                       convLayers.out_units))
for i, (X, _) in enumerate(dl_train_cifar100):
    idx_start = BATCH_SIZE * i
    idx_end = idx_start + X.shape[0]
    outputs[idx_start:idx_end] = torch.flatten(convLayers(X), start_dim=1)

In [ ]:
means = outputs.mean(dim=0)
stds = outputs.std(dim=0)

In [ ]:
torch.set_printoptions(threshold=10_000)
print(means.detach())
print(stds.detach())

tensor([0.3675, 0.3050, 0.3678, 0.3993, 0.2538, 0.4292, 0.4093, 0.4162, 0.1650,
        0.2584, 0.3470, 0.6910, 0.3170, 0.3576, 0.3422, 0.5355, 0.3403, 0.4007,
        0.4311, 0.3835, 0.2333, 0.3313, 0.3861, 0.5779, 0.3348, 0.3360, 0.2470,
        0.5728, 0.2147, 0.2532, 0.5708, 0.5216, 0.1153, 0.3762, 0.3573, 0.6760,
        0.3503, 0.3183, 0.3918, 0.4665, 0.2565, 0.4752, 0.2887, 0.4796, 0.2507,
        0.3514, 0.3140, 0.5348, 0.2286, 0.2614, 0.3718, 0.6396, 0.3978, 0.4147,
        0.4028, 0.3256, 0.3294, 0.4257, 0.3350, 0.4256, 0.2804, 0.5108, 0.3273,
        0.3295, 0.3815, 0.4749, 0.2431, 0.3955, 0.2424, 0.3218, 0.3497, 0.5949,
        0.3411, 0.3484, 0.4228, 0.4088, 0.2468, 0.3322, 0.5202, 0.4392, 0.2458,
        0.2628, 0.5270, 0.5120, 0.2775, 0.2773, 0.5170, 0.4215, 0.3596, 0.5184,
        0.3428, 0.3916, 0.2917, 0.5244, 0.3561, 0.3683, 0.3314, 0.2984, 0.2974,
        0.5779, 0.2083, 0.2376, 0.3983, 0.6485, 0.3362, 0.3510, 0.3520, 0.4940,
        0.2201, 0.4031, 0.3250, 0.5962, 

```
mean = tensor([0.3675, 0.3050, 0.3678, 0.3993, 0.2538, 0.4292, 0.4093, 0.4162, 0.1650,
        0.2584, 0.3470, 0.6910, 0.3170, 0.3576, 0.3422, 0.5355, 0.3403, 0.4007,
        0.4311, 0.3835, 0.2333, 0.3313, 0.3861, 0.5779, 0.3348, 0.3360, 0.2470,
        0.5728, 0.2147, 0.2532, 0.5708, 0.5216, 0.1153, 0.3762, 0.3573, 0.6760,
        0.3503, 0.3183, 0.3918, 0.4665, 0.2565, 0.4752, 0.2887, 0.4796, 0.2507,
        0.3514, 0.3140, 0.5348, 0.2286, 0.2614, 0.3718, 0.6396, 0.3978, 0.4147,
        0.4028, 0.3256, 0.3294, 0.4257, 0.3350, 0.4256, 0.2804, 0.5108, 0.3273,
        0.3295, 0.3815, 0.4749, 0.2431, 0.3955, 0.2424, 0.3218, 0.3497, 0.5949,
        0.3411, 0.3484, 0.4228, 0.4088, 0.2468, 0.3322, 0.5202, 0.4392, 0.2458,
        0.2628, 0.5270, 0.5120, 0.2775, 0.2773, 0.5170, 0.4215, 0.3596, 0.5184,
        0.3428, 0.3916, 0.2917, 0.5244, 0.3561, 0.3683, 0.3314, 0.2984, 0.2974,
        0.5779, 0.2083, 0.2376, 0.3983, 0.6485, 0.3362, 0.3510, 0.3520, 0.4940,
        0.2201, 0.4031, 0.3250, 0.5962, 0.2458, 0.4624, 0.2977, 0.4625, 0.2393,
        0.3893, 0.4570, 0.4106, 0.2311, 0.3242, 0.3973, 0.5477, 0.3083, 0.3222,
        0.3153, 0.5851, 0.2774, 0.4046, 0.3503, 0.4639, 0.2704, 0.3547, 0.4496,
        0.4492, 0.1641, 0.4168, 0.4375, 0.4814, 0.2445, 0.3048, 0.3922, 0.5792,
        0.3887, 0.3398, 0.5124, 0.2511, 0.2899, 0.3037, 0.5047, 0.4217, 0.4087,
        0.4309, 0.2946, 0.3665, 0.2669, 0.3530, 0.4362, 0.4530, 0.1802, 0.3968,
        0.3970, 0.5619, 0.3495, 0.4469, 0.4232, 0.2852, 0.3426, 0.3701, 0.5616,
        0.2972, 0.5472, 0.2994, 0.3388, 0.2664, 0.2332, 0.3866, 0.4384, 0.4611,
        0.2501, 0.3021, 0.4890, 0.4397, 0.2597, 0.3486, 0.2993, 0.5596, 0.1722,
        0.2097, 0.3983, 0.7399, 0.2700, 0.3666, 0.4715, 0.4151, 0.2738, 0.3384,
        0.3116, 0.5879, 0.3491, 0.4454, 0.4258, 0.3497, 0.3474, 0.3966, 0.4309,
        0.3880, 0.3449, 0.2072, 0.5254, 0.4214, 0.3208, 0.3281, 0.5194, 0.3229,
        0.4240, 0.3713, 0.2933, 0.3967, 0.3335, 0.3955, 0.3947, 0.3976, 0.3361,
        0.4201, 0.3274, 0.3588, 0.1935, 0.4248, 0.3474, 0.5374, 0.3558, 0.2820,
        0.4120, 0.4761, 0.1632, 0.4637, 0.2848, 0.5530, 0.3021, 0.4440, 0.2741,
        0.4592, 0.2538, 0.4380, 0.3248, 0.5027, 0.1965, 0.4129, 0.2643, 0.5940,
        0.3808, 0.2956, 0.4113, 0.4647, 0.2247, 0.2939, 0.3555, 0.6432, 0.3224,
        0.4532, 0.2764, 0.4643, 0.3943, 0.3448, 0.4232, 0.4200, 0.3059, 0.3518,
        0.3859, 0.4835, 0.2660, 0.5323, 0.3337, 0.3396, 0.2522, 0.4095, 0.4110,
        0.4500, 0.2124, 0.3367, 0.3934, 0.5415, 0.2347, 0.3556, 0.3550, 0.5817,
        0.2330, 0.4931, 0.2911, 0.5310, 0.2347, 0.4638, 0.3036, 0.5038, 0.2934,
        0.5281, 0.3445, 0.3642, 0.2757, 0.3068, 0.4113, 0.5073, 0.4152, 0.3743,
        0.3648, 0.3743, 0.1987, 0.3851, 0.3322, 0.6074, 0.2809, 0.4795, 0.2335,
        0.5412, 0.2280, 0.3809, 0.3264, 0.5438, 0.2792, 0.4042, 0.3903, 0.4653,
        0.1529, 0.2873, 0.3680, 0.7174, 0.1431, 0.2629, 0.3795, 0.7409, 0.2071,
        0.2284, 0.4540, 0.5855, 0.2550, 0.3618, 0.4102, 0.4964, 0.1965, 0.3238,
        0.4275, 0.5758, 0.2707, 0.3829, 0.4495, 0.4435, 0.1589, 0.3488, 0.4204,
        0.5570, 0.1666, 0.3422, 0.3001, 0.6535, 0.2552, 0.4455, 0.3413, 0.4975,
        0.1963, 0.3399, 0.3873, 0.5543, 0.2302, 0.5799, 0.3400, 0.3501, 0.2753,
        0.4078, 0.3944, 0.3474, 0.1354, 0.3172, 0.4489, 0.6235, 0.2297, 0.3563,
        0.3538, 0.5818, 0.2245, 0.4660, 0.3268, 0.4621, 0.2731, 0.4787, 0.3329,
        0.4015, 0.1737, 0.2753, 0.5420, 0.5189, 0.2698, 0.3572, 0.3816, 0.5258,
        0.3019, 0.2482, 0.5899, 0.4110, 0.2006, 0.2996, 0.5069, 0.5139, 0.3199,
        0.3927, 0.3559, 0.4340, 0.2508, 0.3962, 0.3303, 0.5352, 0.2117, 0.4766,
        0.2119, 0.5993, 0.1759, 0.4007, 0.3320, 0.5669, 0.2937, 0.3528, 0.2860,
        0.5061, 0.2781, 0.3565, 0.3413, 0.4979, 0.3225, 0.3920, 0.2517, 0.5967,
        0.1434, 0.2153, 0.2983, 0.7763, 0.3020, 0.4954, 0.2394, 0.5180, 0.2417,
        0.4211, 0.2792, 0.5158, 0.2227, 0.3235, 0.3705, 0.5859, 0.2675, 0.3804,
        0.3675, 0.4421, 0.2261, 0.3338, 0.2997, 0.6021, 0.1368, 0.3264, 0.4397,
        0.6400, 0.3135, 0.3820, 0.3828, 0.4229, 0.2788, 0.5010, 0.2708, 0.4571,
        0.2772, 0.4681, 0.2414, 0.5507, 0.2972, 0.4050, 0.3172, 0.5511, 0.2057,
        0.4035, 0.2831, 0.5548, 0.2258, 0.3103, 0.3273, 0.6281, 0.3552, 0.3554,
        0.3100, 0.4586, 0.2574, 0.3006, 0.3684, 0.5848, 0.2209, 0.4859, 0.4237,
        0.3898, 0.2249, 0.3387, 0.3313, 0.6205, 0.2419, 0.4537, 0.3517, 0.4504,
        0.1302, 0.2838, 0.3532, 0.7224, 0.3296, 0.3205, 0.4238, 0.4400, 0.2345,
        0.5518, 0.2138, 0.4992, 0.1929, 0.5347, 0.2627, 0.5044, 0.3259, 0.2220,
        0.4199, 0.5078, 0.1287, 0.2538, 0.4324, 0.7098, 0.3226, 0.4305, 0.2864,
        0.4420, 0.3343, 0.2949, 0.3721, 0.4554, 0.1933, 0.3434, 0.3491, 0.6373,
        0.2105, 0.2595, 0.5203, 0.5622, 0.3881, 0.4895, 0.2337, 0.4419, 0.1365,
        0.4391, 0.2397, 0.6495, 0.3823, 0.2711, 0.5025, 0.3544, 0.2788, 0.3328,
        0.3601, 0.5540, 0.3518, 0.2806, 0.5003, 0.4018, 0.2138, 0.2883, 0.3818,
        0.6406, 0.4418, 0.4369, 0.2840, 0.3425, 0.0960, 0.3862, 0.2832, 0.6798,
        0.2244, 0.3354, 0.3663, 0.5673, 0.2135, 0.2118, 0.4910, 0.5998, 0.2543,
        0.5104, 0.2739, 0.5056, 0.2582, 0.3610, 0.3438, 0.5909, 0.2267, 0.5583,
        0.2803, 0.4297, 0.1706, 0.4326, 0.3044, 0.5805, 0.1800, 0.3091, 0.3739,
        0.6144, 0.1745, 0.3146, 0.3031, 0.6625, 0.1896, 0.3586, 0.4452, 0.5326,
        0.2731, 0.4436, 0.3866, 0.4025, 0.3238, 0.4305, 0.2818, 0.4320, 0.2509,
        0.3809, 0.3945, 0.5173, 0.3716, 0.3729, 0.4734, 0.3164, 0.2119, 0.3124,
        0.3671, 0.6494, 0.3902, 0.4101, 0.3466, 0.3672, 0.2933, 0.3843, 0.3245,
        0.4963, 0.2129, 0.3145, 0.3915, 0.5883, 0.3335, 0.4184, 0.2701, 0.4148,
        0.4067, 0.3551, 0.4651, 0.2643, 0.2547, 0.3497, 0.4690, 0.4378, 0.3467,
        0.3383, 0.4200, 0.3440, 0.1604, 0.2803, 0.3387, 0.6416, 0.3353, 0.3806,
        0.3865, 0.3978, 0.1843, 0.5873, 0.2535, 0.5223, 0.2582, 0.4108, 0.3249,
        0.5223, 0.2657, 0.3181, 0.4636, 0.4536, 0.3602, 0.3970, 0.3471, 0.4311,
        0.2766, 0.4491, 0.3637, 0.4508, 0.2449, 0.3929, 0.4427, 0.3891, 0.3492,
        0.5455, 0.2632, 0.3701, 0.2361, 0.3759, 0.4162, 0.4515, 0.2896, 0.4531,
        0.3526, 0.4214, 0.2519, 0.5443, 0.2575, 0.4957, 0.3485, 0.5633, 0.2263,
        0.3837, 0.2627, 0.4827, 0.2588, 0.5052, 0.2184, 0.4158, 0.3610, 0.5219,
        0.1860, 0.4322, 0.2602, 0.6294, 0.2290, 0.3552, 0.4380, 0.5437, 0.2631,
        0.3306, 0.3734, 0.5240, 0.1756, 0.4843, 0.2398, 0.6461, 0.2683, 0.4601,
        0.3342, 0.3618, 0.2893, 0.3749, 0.3105, 0.5106, 0.2223, 0.3705, 0.3209,
        0.5997, 0.3533, 0.3941, 0.3577, 0.4144, 0.1338, 0.3095, 0.3773, 0.6487,
        0.1824, 0.4985, 0.1669, 0.6371, 0.3158, 0.3138, 0.4867, 0.4468, 0.2857,
        0.3877, 0.3055, 0.5237, 0.2023, 0.2647, 0.4915, 0.6021, 0.3474, 0.3238,
        0.4414, 0.3589, 0.2628, 0.2716, 0.4683, 0.4999, 0.1635, 0.3932, 0.4386,
        0.5270, 0.2198, 0.3481, 0.3464, 0.5918, 0.1411, 0.2676, 0.4625, 0.6099,
        0.2648, 0.3906, 0.3201, 0.5367, 0.3066, 0.3909, 0.3366, 0.4700, 0.2357,
        0.4158, 0.3353, 0.5108, 0.3206, 0.3531, 0.4185, 0.4026, 0.2359, 0.3010,
        0.4866, 0.5023, 0.2693, 0.3404, 0.4984, 0.4078, 0.3072, 0.4133, 0.3735,
        0.3900, 0.2053, 0.3418, 0.5189, 0.3759, 0.2290, 0.4537, 0.4190, 0.3832,
        0.4627, 0.4014, 0.3410, 0.2933, 0.3236, 0.3965, 0.3026, 0.4497, 0.2416,
        0.3738, 0.3255, 0.5486, 0.3002, 0.3146, 0.3391, 0.4696, 0.2550, 0.4478,
        0.3345, 0.5319, 0.1343, 0.3242, 0.3403, 0.7253, 0.2882, 0.4678, 0.2059,
        0.5974, 0.1988, 0.3668, 0.3482, 0.6388, 0.3915, 0.4422, 0.1785, 0.4995,
        0.2959, 0.2979, 0.4002, 0.5158, 0.3728, 0.4743, 0.2791, 0.3193, 0.2624,
        0.3585, 0.2904, 0.5964, 0.2696, 0.5376, 0.2751, 0.4442, 0.2801, 0.4069,
        0.4612, 0.4019, 0.3851, 0.4015, 0.2963, 0.4155, 0.0911, 0.3310, 0.3316,
        0.7052, 0.3886, 0.2954, 0.4135, 0.3809, 0.2706, 0.2621, 0.4282, 0.5280,
        0.2815, 0.3259, 0.3594, 0.5229, 0.2309, 0.4259, 0.3181, 0.5487, 0.2368,
        0.4158, 0.3996, 0.4102, 0.2833, 0.3622, 0.4941, 0.3564, 0.3401, 0.4712,
        0.2125, 0.3655, 0.2089, 0.4452, 0.3529, 0.4806, 0.2869, 0.4514, 0.4126,
        0.3812, 0.2724, 0.3067, 0.4448, 0.5166, 0.3647, 0.3147, 0.4286, 0.4160,
        0.2524, 0.2882, 0.3842, 0.5691, 0.2584, 0.2181, 0.5147, 0.5632, 0.1986,
        0.3299, 0.4232, 0.5186, 0.2234, 0.3818, 0.4580, 0.4255, 0.3057, 0.3312,
        0.4415, 0.3968, 0.2270, 0.2349, 0.5071, 0.4901, 0.1892, 0.4207, 0.3863,
        0.5096, 0.2017, 0.4337, 0.4307, 0.4183, 0.2401, 0.3544, 0.4073, 0.4880,
        0.1655, 0.3768, 0.3508, 0.6572, 0.3064, 0.4310, 0.4019, 0.3770, 0.2727,
        0.2624, 0.3971, 0.5498, 0.3317, 0.4054, 0.3116, 0.4128, 0.3936, 0.3966,
        0.3938, 0.3753, 0.2107, 0.3960, 0.4629, 0.4429, 0.2961, 0.2619, 0.4878,
        0.5226, 0.2579, 0.2244, 0.4476, 0.5852, 0.1948, 0.4438, 0.3188, 0.4846,
        0.4288, 0.2937, 0.2201, 0.5535, 0.2561, 0.2256, 0.5222, 0.4517, 0.2225,
        0.3601, 0.2640, 0.6771, 0.2370, 0.3911, 0.2721, 0.5984])
    ```

```
std = tensor([0.4398, 0.4750, 0.5123, 0.6597, 0.4182, 0.6236, 0.5715, 0.7204, 0.3099,
        0.4493, 0.5195, 0.8383, 0.4307, 0.5671, 0.5473, 0.8218, 0.5119, 0.6287,
        0.5976, 0.6591, 0.4200, 0.5283, 0.5703, 0.8451, 0.4953, 0.5079, 0.4767,
        0.8968, 0.3680, 0.5128, 0.7471, 0.7861, 0.3105, 0.5800, 0.6173, 0.9009,
        0.4914, 0.5294, 0.5845, 0.7723, 0.3952, 0.6229, 0.4748, 0.7400, 0.4255,
        0.5380, 0.5511, 0.7974, 0.3695, 0.4469, 0.5482, 0.8296, 0.4897, 0.5908,
        0.5516, 0.6123, 0.5164, 0.6371, 0.5538, 0.6959, 0.4192, 0.6199, 0.5387,
        0.5984, 0.5073, 0.6080, 0.4878, 0.6763, 0.3823, 0.5172, 0.5456, 0.8203,
        0.4851, 0.5411, 0.5980, 0.6947, 0.4333, 0.6013, 0.7118, 0.7507, 0.4226,
        0.4806, 0.7255, 0.7778, 0.4018, 0.5020, 0.6220, 0.7151, 0.4594, 0.6740,
        0.5139, 0.7171, 0.4419, 0.6351, 0.5944, 0.6282, 0.4978, 0.5094, 0.5392,
        0.8262, 0.3802, 0.4570, 0.5884, 0.8747, 0.4794, 0.5689, 0.5558, 0.7759,
        0.3986, 0.6292, 0.5360, 0.8234, 0.3914, 0.6043, 0.5283, 0.7545, 0.4101,
        0.5764, 0.6580, 0.7268, 0.4108, 0.5136, 0.5787, 0.7668, 0.4354, 0.5301,
        0.5524, 0.8490, 0.3958, 0.5575, 0.5572, 0.7685, 0.4317, 0.5797, 0.6438,
        0.7552, 0.3201, 0.5842, 0.6221, 0.7669, 0.3971, 0.4869, 0.5815, 0.7816,
        0.4386, 0.5021, 0.5979, 0.4912, 0.4543, 0.5050, 0.6636, 0.7154, 0.5040,
        0.5915, 0.5038, 0.6512, 0.4261, 0.6033, 0.5915, 0.7352, 0.3340, 0.5605,
        0.5895, 0.8379, 0.4368, 0.6304, 0.5484, 0.5938, 0.4996, 0.5795, 0.6899,
        0.5727, 0.6460, 0.5278, 0.5436, 0.5244, 0.3777, 0.5872, 0.6243, 0.7545,
        0.4162, 0.5191, 0.6457, 0.6892, 0.3855, 0.5392, 0.5197, 0.8226, 0.3478,
        0.4348, 0.6240, 0.9137, 0.4432, 0.5446, 0.6230, 0.6927, 0.4001, 0.5089,
        0.5487, 0.8155, 0.4923, 0.5962, 0.5848, 0.6455, 0.4993, 0.5791, 0.5887,
        0.6380, 0.4923, 0.4234, 0.6498, 0.7074, 0.4331, 0.4998, 0.6157, 0.5665,
        0.4723, 0.5303, 0.4694, 0.6402, 0.5075, 0.6334, 0.5818, 0.7143, 0.4417,
        0.5622, 0.5307, 0.6527, 0.3527, 0.5829, 0.5412, 0.8569, 0.4472, 0.4980,
        0.5870, 0.7486, 0.2934, 0.5556, 0.4734, 0.7729, 0.4106, 0.6346, 0.4939,
        0.7141, 0.3888, 0.5805, 0.5437, 0.7419, 0.3525, 0.5749, 0.4681, 0.8853,
        0.5101, 0.5332, 0.6116, 0.7672, 0.3572, 0.4798, 0.5384, 0.8595, 0.4396,
        0.6039, 0.4958, 0.7512, 0.4813, 0.5545, 0.5749, 0.7104, 0.4782, 0.5377,
        0.6601, 0.8146, 0.4278, 0.6741, 0.5226, 0.6050, 0.4007, 0.6096, 0.5929,
        0.7286, 0.4356, 0.5525, 0.6035, 0.7892, 0.3972, 0.5563, 0.5368, 0.8188,
        0.4233, 0.6317, 0.5232, 0.7534, 0.3947, 0.6696, 0.5103, 0.7510, 0.4166,
        0.6543, 0.5181, 0.6301, 0.3721, 0.4788, 0.5891, 0.7402, 0.5507, 0.5887,
        0.5784, 0.6798, 0.3973, 0.5942, 0.5559, 0.8494, 0.4434, 0.6675, 0.4638,
        0.8313, 0.3559, 0.5452, 0.5382, 0.8104, 0.4454, 0.6368, 0.6015, 0.7285,
        0.3328, 0.5037, 0.5897, 0.9454, 0.3220, 0.4987, 0.5947, 0.9695, 0.3413,
        0.4201, 0.5984, 0.7965, 0.3983, 0.5865, 0.6133, 0.7763, 0.4013, 0.5675,
        0.6544, 0.8495, 0.4608, 0.5848, 0.6109, 0.7047, 0.2875, 0.4920, 0.5677,
        0.7396, 0.3335, 0.5522, 0.5299, 0.8884, 0.3841, 0.5734, 0.5276, 0.7242,
        0.3816, 0.5534, 0.6158, 0.8165, 0.3940, 0.6770, 0.5354, 0.6222, 0.3800,
        0.5624, 0.5744, 0.6437, 0.3245, 0.5660, 0.6451, 0.8454, 0.4113, 0.5670,
        0.5646, 0.8494, 0.3769, 0.6374, 0.5410, 0.7479, 0.4121, 0.6425, 0.5235,
        0.6713, 0.3309, 0.4755, 0.7378, 0.8078, 0.4579, 0.5947, 0.6419, 0.7904,
        0.4666, 0.4644, 0.6983, 0.7109, 0.3407, 0.5051, 0.6240, 0.7645, 0.4594,
        0.5604, 0.5398, 0.7112, 0.3878, 0.6251, 0.5271, 0.8341, 0.3653, 0.6097,
        0.4345, 0.7863, 0.3166, 0.5793, 0.5349, 0.8116, 0.4573, 0.5344, 0.5040,
        0.7672, 0.3985, 0.5227, 0.5385, 0.7490, 0.4497, 0.5890, 0.4561, 0.8253,
        0.2942, 0.4105, 0.4806, 0.8982, 0.4682, 0.6330, 0.4414, 0.7654, 0.3800,
        0.6017, 0.5180, 0.7884, 0.4027, 0.5552, 0.5943, 0.8467, 0.4044, 0.5412,
        0.5460, 0.6996, 0.3546, 0.5250, 0.5635, 0.8588, 0.2932, 0.5512, 0.6031,
        0.8306, 0.5004, 0.6023, 0.5842, 0.7228, 0.4399, 0.6668, 0.5001, 0.7732,
        0.4533, 0.6309, 0.4654, 0.7866, 0.4677, 0.6009, 0.5370, 0.8166, 0.3724,
        0.5713, 0.4797, 0.7871, 0.3960, 0.5272, 0.5396, 0.8469, 0.4831, 0.5587,
        0.5208, 0.7505, 0.4314, 0.5383, 0.5778, 0.8218, 0.3686, 0.6196, 0.5852,
        0.6819, 0.4131, 0.5709, 0.5540, 0.8564, 0.4516, 0.6577, 0.6114, 0.7097,
        0.2931, 0.4823, 0.5452, 0.9207, 0.4906, 0.5168, 0.6117, 0.7467, 0.4177,
        0.7059, 0.4505, 0.7807, 0.3499, 0.7118, 0.4547, 0.7707, 0.4669, 0.4356,
        0.6181, 0.7814, 0.2870, 0.4625, 0.6056, 0.8802, 0.4581, 0.5720, 0.5153,
        0.7168, 0.4556, 0.4840, 0.5562, 0.7186, 0.3935, 0.5694, 0.5999, 0.8869,
        0.3971, 0.5331, 0.6884, 0.8358, 0.5596, 0.6520, 0.4664, 0.6938, 0.2858,
        0.5635, 0.4680, 0.8336, 0.4491, 0.4723, 0.6247, 0.6484, 0.4122, 0.5279,
        0.5656, 0.8259, 0.4680, 0.4658, 0.6628, 0.6680, 0.4129, 0.4758, 0.5714,
        0.8521, 0.4526, 0.5441, 0.4454, 0.5783, 0.2293, 0.5427, 0.5019, 0.8508,
        0.3943, 0.5412, 0.5882, 0.8186, 0.3625, 0.4098, 0.6743, 0.8205, 0.4182,
        0.6899, 0.4953, 0.7966, 0.4129, 0.5434, 0.5738, 0.8630, 0.3692, 0.6455,
        0.4677, 0.6907, 0.3135, 0.6087, 0.4929, 0.8140, 0.3746, 0.5035, 0.6294,
        0.8945, 0.3663, 0.5726, 0.5048, 0.9353, 0.3644, 0.5687, 0.6496, 0.7842,
        0.4752, 0.6487, 0.6196, 0.7126, 0.4404, 0.6123, 0.4578, 0.6647, 0.4752,
        0.6008, 0.6277, 0.7628, 0.4600, 0.5519, 0.6092, 0.5903, 0.4042, 0.5336,
        0.6000, 0.8612, 0.4599, 0.5703, 0.5276, 0.6161, 0.4669, 0.5461, 0.5570,
        0.7419, 0.3916, 0.5234, 0.6062, 0.8668, 0.4001, 0.5610, 0.4612, 0.6718,
        0.4657, 0.5400, 0.5844, 0.5230, 0.3700, 0.5343, 0.6304, 0.7023, 0.4223,
        0.4988, 0.5352, 0.6119, 0.3135, 0.4607, 0.5322, 0.8567, 0.4641, 0.5929,
        0.5751, 0.6712, 0.3655, 0.6876, 0.4681, 0.7619, 0.3863, 0.5739, 0.5317,
        0.7800, 0.4697, 0.5564, 0.6426, 0.7497, 0.4682, 0.5835, 0.5640, 0.7369,
        0.4433, 0.6260, 0.5876, 0.7437, 0.3915, 0.5265, 0.5682, 0.6400, 0.4450,
        0.7074, 0.4519, 0.6411, 0.3872, 0.5861, 0.6231, 0.7253, 0.4339, 0.5982,
        0.5251, 0.6769, 0.3996, 0.6538, 0.4773, 0.7536, 0.4622, 0.6461, 0.4486,
        0.7352, 0.4662, 0.6566, 0.4993, 0.7765, 0.3831, 0.6452, 0.5786, 0.8012,
        0.3795, 0.6781, 0.4819, 0.8760, 0.4063, 0.5865, 0.6428, 0.8246, 0.4264,
        0.5062, 0.5907, 0.8131, 0.3365, 0.6612, 0.4577, 0.9256, 0.4203, 0.6536,
        0.5043, 0.6592, 0.3979, 0.5541, 0.4965, 0.7973, 0.4193, 0.6142, 0.5507,
        0.8706, 0.4707, 0.5425, 0.5704, 0.6895, 0.3167, 0.5467, 0.5877, 0.8708,
        0.3191, 0.6493, 0.3799, 0.8525, 0.4377, 0.5371, 0.6541, 0.7358, 0.4301,
        0.5361, 0.5431, 0.7625, 0.3851, 0.4887, 0.6559, 0.8229, 0.4461, 0.4983,
        0.5858, 0.6309, 0.3806, 0.4489, 0.6277, 0.7704, 0.3327, 0.5859, 0.5966,
        0.7897, 0.3747, 0.5331, 0.5495, 0.8500, 0.3215, 0.4977, 0.6026, 0.8346,
        0.4609, 0.6170, 0.5640, 0.8174, 0.4430, 0.6106, 0.5350, 0.7696, 0.3871,
        0.6415, 0.5156, 0.7840, 0.4655, 0.5766, 0.5856, 0.6950, 0.3898, 0.4674,
        0.6986, 0.7842, 0.4230, 0.5621, 0.6801, 0.6945, 0.4533, 0.5641, 0.5577,
        0.6568, 0.3346, 0.5170, 0.6482, 0.6458, 0.3797, 0.5864, 0.5943, 0.6472,
        0.4870, 0.5659, 0.4925, 0.5700, 0.4720, 0.5769, 0.5347, 0.7306, 0.3813,
        0.5734, 0.4924, 0.7559, 0.4003, 0.4736, 0.5696, 0.7504, 0.4339, 0.6807,
        0.5862, 0.7801, 0.2733, 0.5050, 0.5413, 0.9211, 0.4289, 0.6144, 0.4030,
        0.7951, 0.3933, 0.6123, 0.5699, 0.8795, 0.5399, 0.6225, 0.3974, 0.7980,
        0.4426, 0.5275, 0.5709, 0.7737, 0.4650, 0.6013, 0.5020, 0.6542, 0.3869,
        0.5119, 0.5059, 0.8222, 0.4338, 0.6835, 0.5223, 0.7191, 0.4554, 0.6323,
        0.6395, 0.7140, 0.4722, 0.5995, 0.5080, 0.7061, 0.2168, 0.4894, 0.5259,
        0.8654, 0.5415, 0.4721, 0.5997, 0.6497, 0.4272, 0.4816, 0.6135, 0.8069,
        0.4312, 0.4725, 0.5519, 0.7318, 0.3522, 0.5987, 0.5276, 0.7650, 0.3794,
        0.5397, 0.6046, 0.6840, 0.4697, 0.6409, 0.6656, 0.6847, 0.4353, 0.5850,
        0.4243, 0.6847, 0.3569, 0.5893, 0.5580, 0.7896, 0.4080, 0.5999, 0.5910,
        0.6575, 0.4450, 0.5169, 0.6485, 0.7962, 0.5362, 0.4937, 0.6070, 0.6601,
        0.4059, 0.5163, 0.6506, 0.8626, 0.3883, 0.4235, 0.6664, 0.7894, 0.3559,
        0.4921, 0.6326, 0.7656, 0.3601, 0.5664, 0.6486, 0.7325, 0.4544, 0.5151,
        0.6555, 0.6892, 0.3669, 0.4466, 0.6562, 0.8088, 0.3616, 0.5915, 0.5638,
        0.7487, 0.3427, 0.5963, 0.6026, 0.7089, 0.3993, 0.5378, 0.6379, 0.7453,
        0.3339, 0.5998, 0.5399, 0.8658, 0.4283, 0.5912, 0.5297, 0.6354, 0.4398,
        0.4730, 0.6140, 0.8183, 0.3956, 0.4980, 0.4776, 0.6571, 0.5132, 0.5840,
        0.6212, 0.6661, 0.3727, 0.6205, 0.5914, 0.7198, 0.4676, 0.4814, 0.6935,
        0.8151, 0.4259, 0.4735, 0.6566, 0.8253, 0.3155, 0.5892, 0.5150, 0.7576,
        0.5172, 0.5201, 0.4326, 0.7773, 0.3704, 0.4082, 0.6533, 0.7414, 0.3556,
        0.5218, 0.4617, 0.8320, 0.3786, 0.5972, 0.4795, 0.8564])
    ```